# **TP : Étude des Cartes de Contrôle**

### **Auteur : KHELID Lilya-Nada**

---

## **Objectifs :**
Dans ce notebook, nous allons explorer l’utilisation des **cartes de contrôle** en contrôle statistique des procédés (CSP). L’objectif est de comprendre et d’implémenter les méthodes d’analyse des variations dans un processus et de vérifier s’il est sous contrôle.

Les thématiques principales abordées sont :

1. **Visualisation et interprétation des cartes de contrôle.**  
2. **Construction des cartes $\bar{X}$ et $R$ pour le suivi des procédés.**  
3. **Analyse des limites de contrôle et détection des anomalies.**  
4. **Influence de la taille des échantillons sur la sensibilité des cartes.**

---

### 📥 Importation des librairies

In [1]:
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

/Users/lilya/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 📥 Importation de la data

In [2]:
df = pd.read_csv("data/controle_qualite.csv") #Figure 1
df.head()

,Sample Number,X_bar,R
0,1,34.5,3
1,2,34.2,4
2,3,31.6,4
3,4,31.5,4
4,5,35.0,5


> ## Exercice 1 :

In [3]:
X_double_bar = df["X_bar"].mean()  # Moyenne générale
R_bar = df["R"].mean()  # Moyenne des étendues

### 📊 Facteurs de la Table de Shewhart pour les Cartes de Contrôle

#### **Facteur A₂ : Carte de Contrôle X̄**
-> Permet de calculer les limites de contrôle de la moyenne **X̄** en fonction de l'étendue moyenne **R̄** :

$$ UCL_{\bar{X}} = \bar{X} + A_2 \times \bar{R} $$  
$$ LCL_{\bar{X}} = \bar{X} - A_2 \times \bar{R} $$  

🔹 **Pourquoi utiliser A₂ ?**  
Il ajuste l'influence de l'étendue pour estimer la variabilité de la moyenne.

#### **Facteurs D₃ et D₄ : Carte de Contrôle R**
-> Permettent de calculer les limites de contrôle de l’étendue **R** :

$$ UCL_{R} = D_4 \times \bar{R} $$  
$$ LCL_{R} = D_3 \times \bar{R} $$  

🔹 **Pourquoi utiliser D₃ et D₄ ?**  
- **D₄** définit la **limite supérieure** de l’étendue **R**.  
- **D₃** définit la **limite inférieure**, souvent **0** quand **n ≤ 6**.

#### **📌 Tableau des Facteurs selon la Taille d'Échantillon (n)**

| Taille d'échantillon (n) | A₂   | D₃   | D₄   |
|-------------------------|------|------|------|
| 2                       | 1.88 | 0.000 | 3.267 |
| 3                       | 1.02 | 0.000 | 2.574 |
| 4                       | 0.729 | 0.000 | 2.282 |
| 5                       | 0.577 | 0.000 | 2.114 |
| 6                       | 0.483 | 0.000 | 2.004 |
| 7                       | 0.419 | 0.076 | 1.924 |
| 8                       | 0.373 | 0.136 | 1.864 |
| 9                       | 0.337 | 0.184 | 1.816 |
| 10                      | 0.308 | 0.223 | 1.777 |


In [4]:
# n=4, table de Shewhart)
A2 = 0.729  
D3 = 0.000  
D4 = 2.282  

#--limites de la carte de contrôle--
UCL_X = X_double_bar + A2 * R_bar
LCL_X = X_double_bar - A2 * R_bar
UCL_R = D4 * R_bar
LCL_R = D3 * R_bar

# Carte X_bar 
fig_x_bar = px.line(df, x="Sample Number", y="X_bar", markers=True, title="Carte de contrôle de la moyenne (X̄)")
fig_x_bar.add_hline(y=X_double_bar, line_dash="dash", line_color="blue", annotation_text="Moyenne générale")
fig_x_bar.add_hline(y=UCL_X, line_dash="dash", line_color="red", annotation_text="UCL")
fig_x_bar.add_hline(y=LCL_X, line_dash="dash", line_color="red", annotation_text="LCL")
fig_x_bar.show()

# Carte R 
fig_r = px.line(df, x="Sample Number", y="R", markers=True, title="Carte de contrôle de l’étendue (R)")
fig_r.add_hline(y=R_bar, line_dash="dash", line_color="blue", annotation_text="Moyenne des étendues")
fig_r.add_hline(y=UCL_R, line_dash="dash", line_color="red", annotation_text="UCL")
fig_r.add_hline(y=LCL_R, line_dash="dash", line_color="red", annotation_text="LCL")
fig_r.show()

In [5]:
print(f"Moyenne générale (X̄): {X_double_bar:.2f}")
print(f"Limite de contrôle supérieure (UCL) pour X̄: {UCL_X:.2f}")
print(f"Limite de contrôle inférieure (LCL) pour X̄: {LCL_X:.2f}")
print(f"Moyenne des étendues (R): {R_bar:.2f}")
print(f"Limite de contrôle supérieure (UCL) pour R: {UCL_R:.2f}")
print(f"Limite de contrôle inférieure (LCL) pour R: {LCL_R:.2f}")

Moyenne générale (X̄): 34.00
Limite de contrôle supérieure (UCL) pour X̄: 37.44
Limite de contrôle inférieure (LCL) pour X̄: 30.57
Moyenne des étendues (R): 4.71
Limite de contrôle supérieure (UCL) pour R: 10.74
Limite de contrôle inférieure (LCL) pour R: 0.00


> ## Exercice 4 :

In [6]:
df = pd.read_csv("data/controle_qualite_1.csv") #Figure 4
df.head()

,Observation,Concentration
0,1,60.4
1,2,69.5
2,3,78.4
3,4,72.8
4,5,78.2


In [7]:
# 1. QQ Plot des Données Brutes
theoretical_quantiles = np.linspace(stats.norm.ppf(0.01), stats.norm.ppf(0.99), len(df))
ordered_values = np.sort(df["Concentration"]) 

fig_qq = go.Figure()
fig_qq.add_trace(go.Scatter(x=theoretical_quantiles, y=ordered_values, mode='markers', name='Données'))
fig_qq.add_trace(go.Scatter(x=theoretical_quantiles, y=theoretical_quantiles * np.std(ordered_values) + np.mean(ordered_values), mode='lines', name='Ligne théorique'))
fig_qq.update_layout(title="QQ Plot des Données Brutes", xaxis_title="Quantiles théoriques", yaxis_title="Quantiles des données")
fig_qq.show()

# 2. Carte d'Étendue Mobile
df["Etendue_mobile"] = df["Concentration"].diff().abs()

fig_mobile = px.line(df[1:], x="Observation", y="Etendue_mobile", markers=True, title="Carte d'Étendue Mobile")
fig_mobile.add_hline(y=df["Etendue_mobile"].mean(), line_dash="dash", line_color="red", annotation_text="Moyenne")
fig_mobile.add_hline(y=df["Etendue_mobile"].mean() + 3 * df["Etendue_mobile"].std(), line_dash="dash", line_color="green", annotation_text="UCL (3σ)")
fig_mobile.add_hline(y=df["Etendue_mobile"].mean() - 3 * df["Etendue_mobile"].std(), line_dash="dash", line_color="green", annotation_text="LCL (3σ)")
fig_mobile.show()

# 3. QQ Plot du Log des Données
df["Log_Concentration"] = np.log(df["Concentration"])  # valeurs > 0
theoretical_quantiles_log = np.linspace(stats.norm.ppf(0.01), stats.norm.ppf(0.99), len(df))
ordered_values_log = np.sort(df["Log_Concentration"]) 

fig_qq_log = go.Figure()
fig_qq_log.add_trace(go.Scatter(x=theoretical_quantiles_log, y=ordered_values_log, mode='markers', name='Données Log'))
fig_qq_log.add_trace(go.Scatter(x=theoretical_quantiles_log, y=theoretical_quantiles_log * np.std(ordered_values_log) + np.mean(ordered_values_log), mode='lines', name='Ligne théorique'))
fig_qq_log.update_layout(title="QQ Plot du Log des Données", xaxis_title="Quantiles théoriques", yaxis_title="Quantiles des données (Log)")
fig_qq_log.show()

# 4. Carte d'Étendue Mobile pour le Log des Données
df["Etendue_mobile_log"] = df["Log_Concentration"].diff().abs()

fig_mobile_log = px.line(df[1:], x="Observation", y="Etendue_mobile_log", markers=True, title="Carte d'Étendue Mobile du Log des Données")
fig_mobile_log.add_hline(y=df["Etendue_mobile_log"].mean(), line_dash="dash", line_color="red", annotation_text="Moyenne")
fig_mobile_log.add_hline(y=df["Etendue_mobile_log"].mean() + 3 * df["Etendue_mobile_log"].std(), line_dash="dash", line_color="green", annotation_text="UCL (3σ)")
fig_mobile_log.add_hline(y=df["Etendue_mobile_log"].mean() - 3 * df["Etendue_mobile_log"].std(), line_dash="dash", line_color="green", annotation_text="LCL (3σ)")
fig_mobile_log.show()


> ## Exercice 5 :

In [8]:
df = pd.read_csv("data/controle_qualite_2.csv") #Figure 4
df.head()

,Valeurs
0,12
1,15
2,14
3,10
4,18


In [9]:
# 1. Estimation par la moyenne de l'étendue mobile
moving_range = np.abs(np.diff(df["Valeurs"].values))
sigma_moving_avg = np.mean(moving_range) / 1.128  # 1.128 est un facteur de correction pour n=2

# 2. Estimation par s/c4 (c4 pour n=20 est approximativement 0.972)
s = np.std(df["Valeurs"].values, ddof=1)
c4 = 0.972
sigma_s_c4 = s / c4

# 3. Estimation par la médiane de l'étendue mobile
sigma_moving_median = np.median(moving_range) / 1.128

# 4. Estimation par la moyenne de l'étendue mobile avec un écart k (3 ≤ k ≤ 20)
sigma_moving_k_list = []
for k in range(3, 21):
    moving_range_k = np.abs(df["Valeurs"].values[k:] - df["Valeurs"].values[:-k])
    sigma_moving_k = np.mean(moving_range_k) / 1.128
    sigma_moving_k_list.append((k, sigma_moving_k))

results_df = pd.DataFrame({
    "Méthode": [
        "Moyenne de l'étendue mobile",
        "s/c4",
        "Médiane de l'étendue mobile",
    ],
    "Estimation écart type": [
        sigma_moving_avg,
        sigma_s_c4,
        sigma_moving_median,
    ]
})

# Ajout des résultats pour les différentes valeurs de k
for k, sigma_k in sigma_moving_k_list:
    results_df = pd.concat([results_df, pd.DataFrame({"Méthode": [f"Moyenne de l'étendue mobile (k={k})"], "Estimation écart type": [sigma_k]})], ignore_index=True)


fig = px.line(df, y="Valeurs", markers=True, title="Visualisation des données")
fig.update_layout(
    xaxis_title="Index",
    yaxis_title="Valeurs",
    showlegend=True,
    template="plotly_white"
)
fig.show()

results_df

/Users/lilya/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/Users/lilya/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



,Méthode,Estimation écart type
0,Moyenne de l'étendue mobile,2.706234
1,s/c4,2.542061
2,Médiane de l'étendue mobile,2.659574
3,Moyenne de l'étendue mobile (k=3),2.816020
4,Moyenne de l'étendue mobile (k=4),2.327128
5,Moyenne de l'étendue mobile (k=5),2.659574
6,Moyenne de l'étendue mobile (k=6),2.849544
7,Moyenne de l'étendue mobile (k=7),2.318603
8,Moyenne de l'étendue mobile (k=8),2.216312
9,Moyenne de l'étendue mobile (k=9),2.337202


> ## Exercice 6 :

In [10]:
df = pd.read_csv("data/controle_qualite_3.csv") #Figure 4
df.head()

,Observation,Nombre_erreurs
0,1,5
1,2,3
2,3,4
3,4,6
4,5,2


In [11]:
# Calcul de la moyenne et des limites de contrôle
moyenne_c = df["Nombre_erreurs"].mean()
ucl_c = moyenne_c + 3 * np.sqrt(moyenne_c)
lcl_c = max(0, moyenne_c - 3 * np.sqrt(moyenne_c))  # La LCL ne peut pas être négative

# Ajouter les limites au DataFrame
df["Moyenne"] = moyenne_c
df["UCL"] = ucl_c
df["LCL"] = lcl_c

# Tracer la carte c avec Plotly Express
fig_c = px.line(df, x="Observation", y="Nombre_erreurs", markers=True, title="Carte c du nombre d'erreurs")
fig_c.add_scatter(x=df["Observation"], y=df["Moyenne"], mode="lines", name="Moyenne", line=dict(color="green", dash="dash"))
fig_c.add_scatter(x=df["Observation"], y=df["UCL"], mode="lines", name="UCL (3σ)", line=dict(color="red", dash="dash"))
fig_c.add_scatter(x=df["Observation"], y=df["LCL"], mode="lines", name="LCL (3σ)", line=dict(color="blue", dash="dash"))
fig_c.show()

# 📌 Construction de la carte t (loi géométrique)

# Transformation des erreurs en temps entre erreurs (inverse)
df["Temps_entre_erreurs"] = df["Nombre_erreurs"].apply(lambda x: 1/x if x > 0 else 0)

# Calcul des paramètres pour la carte t
moyenne_t = df["Temps_entre_erreurs"].mean()
ucl_t = moyenne_t + 3 * df["Temps_entre_erreurs"].std()
lcl_t = max(0, moyenne_t - 3 * df["Temps_entre_erreurs"].std())

# Ajouter les limites au DataFrame
df["Moyenne_t"] = moyenne_t
df["UCL_t"] = ucl_t
df["LCL_t"] = lcl_t

# Tracer la carte t avec Plotly Express
fig_t = px.line(df, x="Observation", y="Temps_entre_erreurs", markers=True, title="Carte t du nombre d'erreurs (loi géométrique)")
fig_t.add_scatter(x=df["Observation"], y=df["Moyenne_t"], mode="lines", name="Moyenne", line=dict(color="green", dash="dash"))
fig_t.add_scatter(x=df["Observation"], y=df["UCL_t"], mode="lines", name="UCL (3σ)", line=dict(color="red", dash="dash"))
fig_t.add_scatter(x=df["Observation"], y=df["LCL_t"], mode="lines", name="LCL (3σ)", line=dict(color="blue", dash="dash"))
fig_t.show()

> ## Exercice 8 :

In [12]:
n = 100  # Taille de l'échantillon
p_bar = 0.080  # Fraction de non-conformité initiale
UCL_p = 0.161  # Limite supérieure
LCL_p = 0  # Limite inférieure

c_bar = p_bar * n
UCL_c = UCL_p * n
LCL_c = max(LCL_p * n, 0)  # Doit être >= 0


- **Ligne centrale** : 8  
- **Bornes de contrôle** :  
  - Limite supérieure (UCL) : 16  
  - Limite inférieure (LCL) : 0  

Interprétation :  
- Si des points dépassent l’UCL, le processus peut être hors contrôle.  
- Si tous les points restent entre UCL et LCL, le processus est stable.  

In [13]:
# Paramètre de la loi de Poisson
n = 100
lambda_poisson = n * p_bar

# Probabilité que X <= 16 sous loi de Poisson
p_x_leq_16 = stats.poisson.cdf(16, lambda_poisson)

# Erreur de type I (alpha)
alpha = 1 - p_x_leq_16

print(f"Erreur de Type I (alpha) : {alpha:.4f} , {alpha*100:.2f}%")

Erreur de Type I (alpha) : 0.0037 , 0.37%


Erreur de Type I

- **Définition** : Probabilité de détecter un problème alors que le processus est sous contrôle.  
- **Valeur** : 0,37 %, ce qui est faible et normal.  

In [14]:
p_new = 0.2
lambda_poisson_new = n * p_new

# Probabilité que X <= 16 sous la nouvelle loi de Poisson
beta = stats.poisson.cdf(16, lambda_poisson_new)

print(f"Erreur de Type II (beta) : {beta:.4f}, {beta*100:.2f}%")

Erreur de Type II (beta) : 0.2211, 22.11%


Erreur de Type II

- **Définition** : Probabilité de ne pas détecter un problème lorsque le processus change.  
- **Valeur** : 22,1 %, indiquant un risque modéré de ne pas détecter un changement.  

In [15]:
p_detection = 1 - beta
prob_detection_4 = 1 - (1 - p_detection) ** 4

print(f"Probabilité de détecter un changement en au plus 4 observations : {prob_detection_4:.4f} , {prob_detection_4*100:.2f}%")

Probabilité de détecter un changement en au plus 4 observations : 0.9976 , 99.76%


Probabilité de détecter un changement après 4 observations  :
- **Valeur** : 99,76 %, indiquant que le contrôle est efficace après quelques observations.  

In [16]:
np.random.seed(42)
lambda_poisson = 5  
nombre_non_conformites = np.random.poisson(lambda_poisson, 25)

c_bar = np.mean(nombre_non_conformites)
UCL_c = c_bar + 3 * np.sqrt(c_bar)
LCL_c = max(0, c_bar - 3 * np.sqrt(c_bar))

df = pd.DataFrame({"Échantillon": range(1, 26), "Non-conformités": nombre_non_conformites})


fig = px.line(df, x="Échantillon", y="Non-conformités", markers=True, title="Carte de contrôle")
fig.add_hline(y=c_bar, line_dash="dash", line_color="green", annotation_text="Ligne centrale")
fig.add_hline(y=UCL_c, line_dash="dash", line_color="red", annotation_text="UCL")
fig.add_hline(y=LCL_c, line_dash="dash", line_color="blue", annotation_text="LCL")

fig.show()